In [ ]:
# https://breakingcode.wordpress.com/2010/06/29/google-search-python/
# https://github.com/MarioVilas/google
!pip install git+git://github.com/MarioVilas/google.git

In [ ]:
import json

import overpass as op
import pandas as pd
import numpy as np
import csv
import codecs
import pygeoj
import locale
import re

from datetime import datetime
from dateutil.parser import parse

api = op.API(timeout=600)

In [ ]:
#list of valid instance
instances =["determinator for date of periodic occurrence","human"]  

#dizionario da 0 a 1997, per ogni key inserisci l'id'del periodo
setRemoveKey = ["strada","antica","via",
         "corso", "piazza", "viale", "borgata", "provinciale","casale","tetto", "vicolo", "località",
         "colle","ponte", "tetti", "frazione", "sentiero", 
         "di","del","dei", "dello", "della","di", 
         "rocca", "privata", "viale","della", "sentiero", "pista", "rondò","per","circonvallazione","comunale","passeggiata"]
timeSlot={}
for i in range(2017):
    if i >= 0 and i <=999: x=1
    elif i >=1000	and i<=1249: x=2
    elif i >=1250	and i<=1491: x=3
    elif i >=1492	and i<=1599: x=4
    elif i >=1600	and i<=1699: x=5
    elif i >=1700	and i<=1788: x=6
    elif i >=1789	and i<=1830: x=7
    elif i >=1831	and i<=1860: x=8
    elif i >=1861	and i<=1900: x=9
    elif i >=1901	and i<=1920: x=10
    elif i >=1920	and i<=1925: x=11
    elif i >=1925	and i<=1940: x=12
    elif i >=1936	and i<=1940: x=13
    elif i >=1941	and i<=1942: x=14
    elif i >=1943	and i<=1946: x=15
    elif i >=1947	and i<=1948: x=16
    elif i >=1949	and i<=1954: x=17
    elif i >=1955	and i<=1965: x=18
    elif i >=1966	and i<=1978: x=19
    elif i >=1978	and i<=1993: x=20
    elif i >=1993	and i<=1995: x=21
    elif i >=1995	and i<=1997: x=22
    elif i>=1998: x=23    
    timeSlot[i]=x  


In [ ]:
#File manager
def save_file(cityKey,cityJson):
    with open('%s.json' % cityKey, 'w') as f: 
        json.dump(cityJson, f)
        
def open_file(fileName):
    with codecs.open(fileName, encoding='utf-8') as json_data:
        cityJson = json.load(json_data)
        return cityJson
    
def open_fileGeoJson(filename):
    with open(filename)  as json_data:
        cityJson = json.load(json_data)
        return cityJson    

In [ ]:
#Merge data with Wikipedia Links and keep only these streets
def addKeyFromWiki (dicNameKey,wikiList):
    content = []
    for via in wikiList:
            link =  wikiList[via]
            listId = dicNameKey[via.encode("UTF-8")]
            #One entry per Idx (1 street : n Idx)
            for idx in listId: 
                dicBase = {'idx':"",'via':"",'wu':""} 
                dicBase['idx']= idx
                dicBase['via']= via 
                dicBase['wu']= link 
                content.append(dicBase)
    return content

In [ ]:
#add Scrapy info: merge scrapy file: link:dati al file che hai usato per fare query su wikipedia: nome via - via token
#rinominare il campo name di scrapy in wn, aggiungere flag su data se human, capire eventi

def addScrapy(content, scrapyFile):
    local= locale.setlocale(locale.LC_TIME,("en","us")) #for dates
    #creo dizionario di scarpy con key url
    count900 =0
    dicScrapy = {}
    for i in scrapyFile:
        urlWiki = i["wiki_url"]
        dicScrapy[urlWiki] = i
    # ci sono dei link per cui non c'è scrapy print dicScrapy["https://it.wikipedia.org/wiki/Giorgio_Ravaz"] 
    #incrocio i dati con il link key da dizionario
    
    for idx in content:
        link = idx["wu"]
        del idx['wu']  #lo cancello per non avere il duplicato  
        if link in dicScrapy:
            scrapyData = dicScrapy[link]
            if "wiki_url" in scrapyData:
                scrapyData["wu"] = scrapyData.pop("wiki_url")#rinominato campo
            if "name" in scrapyData:
                scrapyData["wn"] = scrapyData.pop("name")#rinominato campo
            if "wd_url" in scrapyData:
                scrapyData["wd"] = scrapyData.pop("wd_url")#rinomino wiki data
            if "img_url" in scrapyData:   
                linkImg = scrapyData["img_url"][0]
                scrapyData.pop("img_url")
                scrapyData["img"] = linkImg
            
            #check su data:
            if scrapyData["instance_type"] in instances:
                #if human or event
                if "date_of_death" in scrapyData:
                    yearM = re.match('.*([1-3][0-9]{3})', scrapyData["date_of_death"])
                    if yearM:
                        year= yearM.group(1)
                        #print year
                        if year>=1900: count900+= 1
                        scrapyData["yd"]=str(year)
                if "date_of_birth" in scrapyData:
                    yearM = re.match('.*([1-3][0-9]{3})', scrapyData["date_of_birth"])
                    if yearM:
                        year= yearM.group(1)
                        scrapyData["yb"]=str(year)  
            idx.update(scrapyData) #dovrebbe mergiare i due dizionari 
            #del idx['wu']  #lo cancello per non avere il duplicato  
    print "COUNT "+str(count900)           
    return content              

In [ ]:
#Keep GeoJson data just for idx with some scrapy data
def dicReducedGeo(gsonFile, content):
    newGeofile = pygeoj.new()
    newGeofile.define_crs(type="link", link="http://spatialreference.org/ref/epsg/26912/esriwkt/", link_type="esriwkt")
            
    for properties in content:  
        idx= str(properties['idx'])
        for feature in gsonFile:
            idxF = feature.properties["id"]
            if idxF == idx:
                newGeofile.add_feature(properties=properties, geometry=feature.geometry)
    
    return newGeofile 


In [ ]:
#Python script to execute from Console
from google import search
import json
import sys

for arg in sys.argv: 1
print arg

city = arg
links = {}
filename = "%s.json" % city
filenameSaved = "W_%s.json" % city
#filename = "TO05.json"

print filename

with open(filename) as json_data:
    wikiTest_short = json.load(json_data)
    for i in wikiTest_short:
        value = wikiTest_short[i].encode('utf-8')
        #check = "Check %s and %s" %(i,wikiTest_short[i])
        #print check
        for url in search(value, tld='it', lang='it', stop=5):
            if url.startswith('https://it.wikipedia.org'):
                links [i] = url
                print url
                break
    #salvami dizionario con chiave la strada e valore il link wikipedia se lo hai trovato
    with open(filenameSaved, 'w') as fp:
        json.dump(links, fp)

In [ ]:
#Keep GeoJson data just for idx with some scrapy data
def dicReducedGeoOnlyInstance(gsonFile, content):
    newGeofile = pygeoj.new()
    newGeofile.define_crs(type="link", link="http://spatialreference.org/ref/epsg/26912/esriwkt/", link_type="esriwkt")
            
    for properties in content:  
        idx= str(properties['idx'])
        #print properties
        if "instance_type" in properties.keys(): #and properties["instance_type"] in instances:
            if properties["instance_type"] in instances:
                for feature in gsonFile:
                    idxF = feature.properties["id"]
                    if idxF == idx:
                        newGeofile.add_feature(properties=properties, geometry=feature.geometry)
    
    return newGeofile 


In [ ]:
#1_ partendo dal geoJson crearsi dic con id e via
cityKey = "Cuneo"  

geoJson = pygeoj.load(filepath="%s.geojson" %cityKey) 
id_name ={}
name_id ={}
for feature in geoJson: #distinti per id
    if feature.properties["name"]:
        name = feature.properties["name"].encode('utf-8')
       
        idx = feature.properties["id"]
        #dict key and name
        id_name[feature.properties["id"]]= name
         
        if name in name_id:    
            lista = name_id[name]
            lista.append(feature.properties["id"])
        else:
            idList = []
            idList.append(feature.properties["id"])
            name_id[name]=idList
            
#print id_name
print len(name_id)

In [ ]:
#clean and token street name in name_idt
wikiAskDic = list()
askGoogleDic = {}
for street in name_id:
        lowerStreet=street.lower()
        streetToken = lowerStreet.split()
        streetTokenL = list(streetToken)
        for x in streetTokenL:
            if x in setRemoveKey:
                streetTokenL.remove(x)
        cleaned = ' '.join(streetTokenL)
        askGoogleDic[street] = cleaned
        wikiAskDic.append(cleaned) 
        
save_file(cityKey+"_askGoogle",askGoogleDic)

In [ ]:
#Carico in wikiList via e url wiki 
with open('W_%s_askGoogle.json' %cityKey) as json_data:
    wikiList = json.load(json_data) 
#print wikiList   
#print name_id

In [ ]:
contentWiki = addKeyFromWiki(name_id,wikiList)
#print contentWiki

In [ ]:
#prendo il file di Scrapy risultante da quello con wiki e creo un json con idx e dictionary con i value
scrapyFile = open_file("%s_dedicationInfo.json" %cityKey)
print len(scrapyFile)
contentWithScrapy = addScrapy(contentWiki,scrapyFile)

In [ ]:
#Alternativa 1 = tengo solo le istanze di human o period occurance

In [ ]:
reducedGeo = dicReducedGeoOnlyInstance(geoJson,contentWithScrapy)
reducedGeo.save("%s_final_Inst.geojson" %cityKey)

In [ ]:
#Alternativa 2 = al geoJson iniziale (che contiene tutte le vie) 
#scremo solo su vie esistenti di ContenWiki

In [ ]:
reducedGeo = dicReducedGeo(geoJson,contentWithScrapy)
reducedGeo.save("%s_final.geojson" %cityKey)


In [ ]:
#FINE